# Remap FOV names for TMAs

NOTE: this notebook should be run after `example_fov_grid_generate.ipynb` <b>for TMA data only</b>. Do not run this notebook for non-TMA data.

One limitation of the commercial instrument is that it's impossible to change FOV definitions once they're entered in. This presents a problem due in part to the nature of how a TMA input file is defined. The grid that the TMA defines leaves a lot of room for variation in between, and it is quite possible that a number of TMA-defined FOVs generated from `example_fov_grid_generate.ipynb` will not appear in the desired location on the TMA.

The goal is to avoid the "commercial instrument deadlock" with bad FOV definitions by being able to adjust these definitions beforehand. It should also be possible to verify the locations of these "redefined FOVs" compared to their TMA-defined FOV counterparts and adjust their names so they match up as closely as possible.

To accomplish this, the script allows you to define a file listing the desired locations of each FOV on your TMA. We'll call these the manual FOVs and the FOVs generated by `example_fov_grid_generate.ipynb` the auto FOVs. Using this script, you can view how closely your manual FOV names match up with the auto FOV names and how close (or far) apart they are from each other. If needed, you can rematch as many manual FOV names as needed to different corresponding auto FOV names prior to remapping your manual FOVs.

In [1]:
import json
import os
from skimage.io import imread

from ark.mibi import tiling_utils

# suppress mpl deprecation
import warnings
from matplotlib.cbook import mplDeprecation
warnings.filterwarnings("ignore", category=mplDeprecation)

### Define the QC parameters and load data

Define the following parameters to set your home directory:

* `base_dir`: the root directory of the tiling data, should be the same as set in `example_fov_grid_generate.ipynb`
* `json_tiling_dir`: the directory in `base_dir` containing the information to read and write the FOV info, should be the same as set in `example_fov_grid_generate.ipynb`

Define the prefix to use for your input and output file names:

* `tma_remap_prefix`: defaults to `example_tma`

Define the following input and output paths (all prefixed by `tma_remap_prefix`):

* `auto_fov_output_path`: the path to the automatically-generated set of FOVs, this is created at the end of `example_fov_grid_generate.ipynb`. Make sure you rename this to the correct output file!
* `manual_fov_output_path`: the path to your proposed set of FOVs. Make sure you rename this to the correct file name!
* `slide_path`: the path to the slide which to take the FOVs
* `mapping_path`: the path which to save the final mapping of your proposed FOVs to the FOVs generated by the script (contained in `augo_fov_regions`)
* `remapped_fov_path`: the path to write the tiles with remapped FOV names
* `moly_path`: the path to the Moly point, needed if you want to insert this between FOVs and/or runs after remapping. Ignored if you choose not to insert Moly points between FOVs.

In [2]:
# define your home directory
base_dir = "../data/example_dataset"
json_tiling_dir = os.path.join(base_dir, "json_tiling_data")

# define the prefix of each file
tma_remap_prefix = 'example_tma'

# define the input and output files
auto_fov_output_path = os.path.join(json_tiling_dir, '%s_fov_output.json' % tma_remap_prefix)
manual_fov_output_path = os.path.join(json_tiling_dir, '%s_proposed_fovs.json' % tma_remap_prefix)
slide_path = os.path.join(json_tiling_dir, '%s_slide.png' % tma_remap_prefix)
mapping_path = os.path.join(json_tiling_dir, '%s_manual_auto_map.json' % tma_remap_prefix)
remapped_fov_path = os.path.join(json_tiling_dir, '%s_noah_test_tiles_remapped.json' % tma_remap_prefix)
moly_path = os.path.join(json_tiling_dir, '%s_moly_point.json' % tma_remap_prefix)

In [3]:
# load the automatically-generated FOVs in
with open(auto_fov_output_path, 'r') as afop:
    auto_fov_regions = json.load(afop)

In [4]:
# load the proposed set of FOVs in
with open(manual_fov_output_path, 'r') as mfop:
    manual_fov_regions = json.load(mfop)

In [5]:
# load the slide image in
slide_data = imread(slide_path)

### Map proposed tiles to their closest automatically-generated tile (Euclidean)

In [6]:
manual_to_auto_map, manual_fovs_info, auto_fovs_info = tiling_utils.assign_closest_fovs(
    manual_fov_regions,
    auto_fov_regions
)

### Visualize and remap tiles

Usage notes:

* Proposed FOVs are drawn in red. Automatically-generated FOVs are drawn in blue.
* The selected proposed FOV and its mapped automatically-generated FOV are colored a darker shade.
* The `Manually-defined FOV` tab can be used to visualize current mappings to automatically-generated FOVs.
* The `Automatically-generated FOV` tab is used for re-mapping a FOV created by this script to one of yours.
* When you're done generating your desired mapping, click the `Save mapping`. This will save your mapping to `mapping_path` specified earlier.

Note:

* The cells after this interactive widget (in the section `Use mapping to rename FOVs in tiled_regions_proposed`) are only to be run after you're satisfied with the mapping and clicked `Save mapping`.
* Prior to clicking `Save mapping` and re-running the cells in that section, ignore any error messages that may appear in that section.

In [7]:
%matplotlib widget
tiling_utils.interactive_remap(
    manual_to_auto_map,
    manual_fovs_info,
    auto_fovs_info,
    slide_data,
    mapping_path,
    draw_radius=7,
    figsize=(7, 7)
)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

Output()

### Use mapping to rename FOVs in `manual_fov_regions`

In [12]:
# load the mapping saved in the interactive visualization in
with open(mapping_path, 'r') as mp:
    mapping = json.load(mp)

In addition to renaming FOVs in `manual_fov_regions`, the following steps can be done if specified:

* The FOVs are randomized (set `randomize = True` in the following cell). 
* Moly points are inserted at a specified interval (set `insert_moly = True` and `moly_interval` to the desired value in the following cell). The same Moly point returned by `set_tiling_params` in step 1 is used.

In [13]:
# whether to randomize the FOVs in remapped_fov_regions
randomize = True

# whether to insert Moly points between a specified interval of FOVs in remapped_fov_regions
moly_insert = True
moly_interval = 5

In [14]:
# rename FOVs, randomize the order, and insert Moly points at a specified interval
remapped_fov_regions = tiling_utils.remap_and_reorder_fovs(
    manual_fov_regions,
    mapping,
    moly_path,
    randomize=randomize,
    moly_insert=moly_insert,
    moly_interval=moly_interval
)

In [15]:
# save remapped_fov_regions with the new FOV names (and randomized and with Moly points inserted if specified)
with open(remapped_fov_path, 'w') as rtp:
    json.dump(remapped_fov_regions, rtp)